In [213]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    OpenAI.api_key = openai_api_key
client = OpenAI()

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import urllib

import time
import datetime as dt

In [18]:
TWITTER_ACCOUNT = os.getenv("TWITTER_ACCOUNT")
TWITTER_PASSWORD = os.getenv("TWITTER_PASSWORD")

In [385]:
def twitter_login():
    driver = webdriver.Chrome()
    driver.get("https://x.com/login/")
    driver.maximize_window()
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "input")))
    driver.find_element(By.TAG_NAME,"input").send_keys(TWITTER_ACCOUNT)
    driver.find_element(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.NAME, "password")))
    driver.find_element(By.NAME,"password").send_keys(TWITTER_PASSWORD)
    driver.find_element(By.XPATH,'//div/span/span[text()="ログイン"]').click()
    return driver
driver = twitter_login()

In [36]:
target_account_name = "hanairoha_ww"
driver.get("https://x.com/{target_account_name}".format(target_account_name=target_account_name))
WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "article")))

<selenium.webdriver.remote.webelement.WebElement (session="9e323b2c3b0ec730c750008f309bc423", element="f.EAEA84D87B740280C78E74A19C8B9FE8.d.F805BE7B8BB4C5F4B6ADD8113C919F1C.e.69")>

In [347]:
#HTMLの構造を見やすくしてくれる関数
def extract_tag_structure(html_content, drop_text=False):
    soup = BeautifulSoup(html_content, "html.parser")

    def clean_element(element):
        element.attrs = {}
        for child in element.contents:
            if child.name:  # タグの場合
                child.attrs = {}  # 属性を削除
                clean_element(child)  # 再帰処理
            elif child.string:  # テキストの場合
                if drop_text:
                    child.extract()  # テキストを削除
    clean_element(soup)
    return soup.prettify()

In [ ]:
def download_file(url, dst_path):#画像の保存
    if os.path.exists(dst_path):
        print("This file exists:",dst_path)
    else:
        try:
            with urllib.request.urlopen(url) as web_file:
                data = web_file.read()
                with open(dst_path, mode='wb') as local_file:
                    local_file.write(data)
        except urllib.error.URLError as e:
            print(e)

In [58]:
#アカウントとツイートIDの組を渡すとツイートオブジェクトを返してくれる関数
def get_tweet_driver(driver, target_account_name, tweet_id):
    driver.get("https://x.com/{account_name}/status/{tweet_id}".format(account_name=target_account_name,tweet_id=tweet_id))
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "article")))
    articles = driver.find_elements(By.TAG_NAME,"article")
    return articles

In [350]:
def get_tweet_info(article, debug_mode=False):#ツイートオブジェクト（article配下）を渡すと要素を出力してくれる関数

    if debug_mode:
        print(article.get_attribute("innerHTML"))
        print("==========")
        print(extract_tag_structure(article.get_attribute("innerHTML")))

    #ツイート単独の画面に行っても、リプ元やおすすめなど関係ないツイートが出てくる。それらは構造が違うので取り除ける
    try:#通常ツイートの場合
        tweet_link = article.find_element(By.XPATH,'div/div/div[3]/div[4]/div/div[1]/div/div[1]/a').get_attribute("href").split("/")#編集履歴がある場合最後に/historyがつく
        account_name = tweet_link[3]
        tweet_id = tweet_link[5]
        tweet_type = "normal"
    except NoSuchElementException:
        try:#リプ元や関連ツイートの場合
            tweet_link = article.find_element(By.XPATH,'div/div/div[2]/div[2]/div/div/div/div/div/div[2]/div/div[3]/a').get_attribute("href").split("/")
            account_name = tweet_link[3]
            tweet_id = tweet_link[5]
            tweet_type = "additional"
        except NoSuchElementException:
            try:#リプ元の場合かつ本体が引用RTの場合（よくわからん）
                tweet_link = article.find_element(By.XPATH,'div/div/div[2]/div[2]/div/div/div/div/div/div[2]/div/div[3]/div/a').get_attribute("href").split("/")
                account_name = tweet_link[3]
                tweet_id = tweet_link[5]#編集履歴がある場合最後に/historyがつくので
                tweet_type = "reply"
            except NoSuchElementException:#ツイート種別不明
                print("ツイート種別不明")
                print(article.get_attribute("innerHTML"))
                print("==========")
                print(extract_tag_structure(article.get_attribute("innerHTML")))
                return {"tweet_id":None, "tweet_type":False, "innerHTML":article.get_attribute("innerHTML")}
    if debug_mode:
        print(tweet_type, account_name, tweet_id)
    if tweet_type!="normal":
        return {"account_name":account_name, "tweet_id":tweet_id, "tweet_type":tweet_type}

    tweet_time = dt.datetime.strftime(
        dt.datetime.strptime(
            article.find_element(By.XPATH,'div/div/div[3]/div[4]').find_element(By.TAG_NAME,'time').get_attribute("datetime")#ツイート日時（世界標準時）
            ,'%Y-%m-%dT%H:%M:%S.%fZ')
        + dt.timedelta(seconds=60*60*9),'%Y%m%d%H%M%S')#日本時間に変更
    if debug_mode:
        print("ツイート日時")
        print(tweet_time)

    tweet_text = article.find_element(By.XPATH,'div/div/div[3]/div[1]').text#ツイート本文
    if debug_mode:
        print("ツイート本文")
        print(tweet_text)

    tweet_appended = article.find_elements(By.XPATH,'div/div/div[3]/div[2]/div/div')#画像、動画、引用元の情報
    photo_links = []
    q_tweet = False
    media_embedded_link = False
    for i, element_appended in enumerate(tweet_appended):
        try:
            if element_appended.find_element(By.XPATH,'div/span').text=="引用":#引用RT
                if debug_mode:
                    print("引用RT")
                try:
                    q_tweet_link = element_appended.find_element(By.TAG_NAME,'a').get_attribute("href").split("/")
                    q_account_name = q_tweet_link[3]
                    q_tweet_id = q_tweet_link[5]
                    q_tweet={"q_tweet_id":q_tweet_id,"q_account_name":q_account_name}
                    if debug_mode:
                        print(q_account_name,q_tweet_id)
                except NoSuchElementException:
                    print("引用RTのTweetID不明")
                    q_tweet={"q_tweet_id":False,"q_account_name":False}
                    # print(element_appended.get_attribute("innerHTML"))
                # print(element_appended.get_attribute("innerHTML"))
            else:#想定外
                print("想定していないケース")
                print(element_appended.get_attribute("innerHTML"))
        except NoSuchElementException:
            try:#外部リンク埋め込み
                media_embedded_link = element_appended.find_element(By.XPATH,'div/a').get_attribute("href")
                if debug_mode:
                    print("外部リンク埋め込み")
                    print(media_embedded_link)
            except NoSuchElementException:#画像・動画
                photo_num=0
                if debug_mode:
                    print("画像・動画")
                # print(element_appended.get_attribute("innerHTML"))

                videos = element_appended.find_elements(By.TAG_NAME,'video')
                if len(videos)>0:
                    if debug_mode:
                        print("動画あり。保存方法不明")
                    for video in videos:
                        if debug_mode:
                            print(video.get_attribute("poster"))
                for photo in element_appended.find_elements(By.TAG_NAME,'a'):
                    photo_num+=1
                    if debug_mode:
                        print("画像","番号:",photo_num)
                    photo_link = photo.find_element(By.TAG_NAME,'img').get_attribute("src").split("&name")[0]
                    if debug_mode:
                        print(photo_link)
                    photo_links.append(photo_link)

            # for element_in_tweet in element_appended.find_elements(By.TAG_NAME,'div'):#過去の産物
                # if element_in_tweet.get_attribute("data-testid")=="tweetPhoto":#article/div/div/div[3]/div[2]/div/div[1]/div/div/div/div/div
                #     videos = element_in_tweet.find_elements(By.TAG_NAME,'video')
                #     for photo in element_in_tweet.find_elements(By.TAG_NAME,'img'):
                #         photo_num+=1
                #         print("画像","番号:",photo_num)
                #         photo_link = photo.get_attribute("src").split("&name")[0]
                #         print(photo_link)
                #         photo_links.append(photo_link)

    return {"account_name":account_name, 
            "tweet_id":tweet_id, 
            "tweet_type":tweet_type, 
            "tweet_time":tweet_time,
            "tweet_text":tweet_text, 
            "photo":len(photo_links)>0,
            "photo_links":photo_links, 
            "media_embedded_link":media_embedded_link,
            "q_tweet":q_tweet
            }


In [351]:
def get_tweet_info_by_id(account_name, tweet_id, debug_mode=False):#アカウント名とツイートIDを指定すると構造化データを出力してくれる
    articles = get_tweet_driver(driver, account_name, tweet_id)
    for article_no, article in enumerate(articles):#ツイート単位
        tweet_data = get_tweet_info(article, debug_mode=debug_mode)
        if tweet_data["tweet_id"]==tweet_id:
            print("tweet_time:",tweet_data["tweet_time"])
            print("tweet_text:",tweet_data["tweet_text"])
            print("photo_links:",tweet_data["photo_links"])
            print("media_embedded_link:",tweet_data["media_embedded_link"])
            if tweet_data["q_tweet"]:
                print("q_tweet:",tweet_data["q_tweet"]["q_tweet_id"])
                #場合によってはRT元の情報も参照してもよい
            return tweet_data
        else:
            pass
            # if article_no==0 or tweet_data["tweet_type"]=="reply":#リプの場合は情報を表示しても良い

- まだ検証しきれていないこと
    - 他人へのリプ、他人の引用
    - 編集履歴のあるツイートの場合

In [352]:
#通常（画像なし）
account_name = "hanairoha_ww"
tweet_id="1850393659541500394"
get_tweet_info_by_id(account_name, tweet_id)

tweet_time: 20241027132641
tweet_text: 特典会についてのご案内

・対バンライブ・単独公演全て共通で特典会の時間は限られております。
様子を見て時間内に終われるよう、チェキ列を早めに締め切らせていただく場合がございます。撮影ご希望の方はお早めにお並びをお願いいたします。

・入場特典に関しまして、基本当日のみ使用可能となりますので、優先的にご利用ください。入場特典は延長不可となりますので使い切れなかった場合は無効となります。
予めご了承ください。

・既に購入済みのチェキ券をお持ちでチェキ列を締め切られた場合、必ず販売デスクにて延長券と交換をお願いします。
チェキ券は当日のみ使用可能となりますので、延長券にしていない過去の日付のチェキ券をお持ちになられたお客様は撮影をお断りさせていただきます。

＜延長券対応につきまして＞
運営都合(チェキ列の締切等)で撮影できなかったチェキ券に関しては延長券と交換させていただきますが、鍵閉め狙い等、故意に使い切れなかったチェキ券は延長券への交換をお断りする場合がございます。
購入後はなるべくお早めにチェキ列に並ばれるようお願いいたします。

花いろははチェキ券・入場特典含めまして2枚までまとめ出しが可能です！
まとめ出しの場合でもトーク時間は枚数分お話できますのでご安心ください。

特典会参加の上でご不明点ございましたらお気軽に販売スタッフにお声掛けくださいませ。
皆様のご協力よろしくお願いいたします。
#花いろは
photo_links: []
media_embedded_link: False


{'account_name': 'hanairoha_ww',
 'tweet_id': '1850393659541500394',
 'tweet_type': 'normal',
 'tweet_time': '20241027132641',
 'tweet_text': '特典会についてのご案内\n\n・対バンライブ・単独公演全て共通で特典会の時間は限られております。\n様子を見て時間内に終われるよう、チェキ列を早めに締め切らせていただく場合がございます。撮影ご希望の方はお早めにお並びをお願いいたします。\n\n・入場特典に関しまして、基本当日のみ使用可能となりますので、優先的にご利用ください。入場特典は延長不可となりますので使い切れなかった場合は無効となります。\n予めご了承ください。\n\n・既に購入済みのチェキ券をお持ちでチェキ列を締め切られた場合、必ず販売デスクにて延長券と交換をお願いします。\nチェキ券は当日のみ使用可能となりますので、延長券にしていない過去の日付のチェキ券をお持ちになられたお客様は撮影をお断りさせていただきます。\n\n＜延長券対応につきまして＞\n運営都合(チェキ列の締切等)で撮影できなかったチェキ券に関しては延長券と交換させていただきますが、鍵閉め狙い等、故意に使い切れなかったチェキ券は延長券への交換をお断りする場合がございます。\n購入後はなるべくお早めにチェキ列に並ばれるようお願いいたします。\n\n花いろははチェキ券・入場特典含めまして2枚までまとめ出しが可能です！\nまとめ出しの場合でもトーク時間は枚数分お話できますのでご安心ください。\n\n特典会参加の上でご不明点ございましたらお気軽に販売スタッフにお声掛けくださいませ。\n皆様のご協力よろしくお願いいたします。\n#花いろは',
 'photo': False,
 'photo_links': [],
 'media_embedded_link': False,
 'q_tweet': False}

In [330]:
#通常（画像1枚）
account_name = "hanairoha_ww"
tweet_id="1850510635991470287"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 【ライブ情報】
11/14(木)
衛星とカラテアPresents
『超 明星現象』

WWWX / WWW 他全8会場予定
前売り4500円
女性2500円 
当日+1000円
(+1D600円 / 再入場自由)

 http://t-dv.com/chomyojo_1114
販売中！
2ショット写メor入場1pt＋30秒トーク延長券

平日のサーキットとなりますが是非会いにきてくださると嬉しいです！
お待ちしております
#花いろは 
#超明星現象
photo_links: ['https://pbs.twimg.com/media/Ga5VmxCbAAAyzvN?format=jpg']
media_embedded_link: False


In [331]:
#通常（画像複数）
account_name = "hanairoha_ww"
tweet_id="1833422444323013007"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 【お知らせ】

今咲ひめの 
@himenonoo
 と
アパレルブランドLIVERTINEAGE 
@livertineage
 さんのコラボ商品の受注販売が開始されました⸜❤︎⸝

商品一覧▽
・ジップパーカー
・ロングTシャツ
・キャップ

販売ページ▽
 https://livertineage.jp/SHOP/204843/239830/list.html…

沢山こだわりが詰まった商品です！
ぜひゲットしてください
photo_links: ['https://pbs.twimg.com/media/GXGf_pZbwAA_W7f?format=jpg', 'https://pbs.twimg.com/media/GXGf_pYasAAUJU0?format=jpg', 'https://pbs.twimg.com/media/GXGf_pUbEAEJtD0?format=jpg', 'https://pbs.twimg.com/media/GXGf_pWbEAI5_K9?format=jpg']
media_embedded_link: False


In [332]:
#通常（動画1つ）
account_name = "hanairoha_ww"
tweet_id="1850445635734196268"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 本日は『TOKYO GIRLS GIRLS Haloween!!』にご来場いただきありがとうございました

メンバーお揃いのSWAT衣装、いかがでしたでしょうか

ハロウィンコスプレは残りあと1回
10/31(木)のみとなりますので、そちらもぜひお見逃しなく
#花いろは
photo_links: []
media_embedded_link: False


In [333]:
#通常（画像動画混在）
account_name = "hanairoha_ww"
tweet_id="1816101043849290094"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 8/14(水) #花いろは 
5thワンマンライブ
『 #一生に一度の夏が来る 』
品川ステラボール

まで  【あと21日】 (3週間)

まきちゃんひまちゃん今夏おすすめアイテム発表！

新曲お披露目あり
http://tiget.net/events/328786
手売りチケット購入特典アリ
#0814品川ステラボール
photo_links: ['https://pbs.twimg.com/media/GTQWJ60aYAMaWOZ?format=jpg']
media_embedded_link: False


In [334]:
#メディア埋め込みリンク系
account_name = "hanairoha_ww"
tweet_id="1847288948353380697"#SHOWROOM
tweet_id="1850506876599079055"#TikTok
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: #花いろは の投稿動画を楽しみましょう！

2人でパフェ作ったよ

#TikTok
photo_links: []
media_embedded_link: https://t.co/Q5qGhLLc5K


In [335]:
#本文＋引用（どちらも画像なし）
account_name = "hanairoha_ww"
tweet_id="1849704587529466353"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 10/19(土)に開催しました『TATSUYA MAKIHO BIRTHDAY LIVE2024〜23歳のまきちゃんも1番かわいいよー！〜』のSチケット特典の缶バッチのご用意ができましたので、本日の物販よりお渡しさせていただきます。

該当される方はお手数ですが、まきほ生誕チケットを表示の上、物販スタッフの方までお声掛けください。
#花いろは
photo_links: []
media_embedded_link: False
q_tweet: 1847539250113446336


In [336]:
#本文＋引用（画像あり）
account_name = "hanairoha_ww"
tweet_id="1850474392192745851"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 10/30(水)に開催いたします、
『-DAIA SHOWCASE- 花いろは お試し無銭ライブ！』ですが、前売りチケット全て完売いたしました！
沢山のお申し込みありがとうございます

そこでAチケットのみ若干枚数追加販売決定しました
本日21時〜受付開始！
悩まれていた方もこの機会にぜひお待ちしております
https://tiget.net/events/355633

✦ː──────────
SHIBUYA DAIA
OPEN19:10 / START19:30
Sチケット(前方・スタンディング)¥2,500
Aチケット(後方・スタンディング) 無料 (各1D必須)
✦ː──────────

当日券は入場状況次第での販売になります。
Aチケット(後方)1000円（別途D代）
を予定しております。
皆様のご来場お待ちしております
#花いろは
photo_links: []
media_embedded_link: False
q_tweet: 1849413237269541237


In [337]:
#本文（画像あり）＋引用（画像あり）
account_name = "hanairoha_ww"
tweet_id="1850479653469008232"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: #花いろは タイテ情報

11/3(日)
『 GIRLS MUSIC SQUARE 
@KYOTO
 』

KYOTO MUSE
開場11:30　開演12:00
Sチケット(前方)3,500円　Aチケット(一般)1,000円　※各1D別
 http://tiget.net/events/355267
┈ ┈ ┈ ┈ ┈ 
13:05-13:30
14:00-15:00(終演後)
┈ ┈ ┈ ┈ ┈ 
2ショット写メor入場2pt

お待ちしております
photo_links: ['https://pbs.twimg.com/media/Ga45b0UaMAAqqS8?format=jpg']
media_embedded_link: False
q_tweet: 1849032422798843953


In [338]:
#本文（動画あり）＋引用（画像なし）
account_name = "hanairoha_ww"
tweet_id="1813205765923987735"
get_tweet_info_by_id(account_name, tweet_id)

引用RTのTweetID不明
tweet_text: 本日は『GIRLS MUSIC SQUARE @ AKIHABARA 〜越智天音誕生日SP〜』ご来場ありがとうございました

次のライブは！
7/18(木)定期公演Vol.80
〜ちょっと遅めの七夕公演〜
SHIBUYA DAIA
開場19:45　開演20:00
http://tiget.net/events/333233

今年初の浴衣公演
お待ちしております #花いろは
photo_links: []
media_embedded_link: False
q_tweet: False


In [339]:
#本文（動画あり）＋引用（画像あり）
account_name = "hanairoha_ww"
tweet_id="1850182080044716529"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 本日は『TOKYO GIRLS GIRLS Haloween!!』にご来場いただきありがとうございました

明日も『TOKYO GIRLS GIRLS Haloween!!』に出演させていただきます！昨日今日とは会場が違うのでご注意ください
┈ ┈ ┈ ┈ ┈ 
EBiS303
前方6000円 通常3000円 当日4000円(+1D)
2ショット写メ＋集合写メor入場1pt
 http://tiget.net/events/348467

13:55-14:20(STAGE A)
14:45-16:05(A)
┈ ┈ ┈ ┈ ┈ 

明日は全員お揃いでSWATのコスプレで登場します
お待ちしております
#花いろは
photo_links: []
media_embedded_link: False
q_tweet: 1848701684895428687


In [340]:
#自分にリプ
account_name = "hanairoha_ww"
tweet_id="1839246897665683848"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: こちら花いろは入場、または物販にてチェキ券1枚以上購入の方にスタンプ押させていただきます。
photo_links: []
media_embedded_link: False


In [341]:
#自分にリプ（画像あり）
account_name = "hanairoha_ww"
tweet_id="1847412471352742171"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 2本目
『TATSUYA MAKIHO BIRTHDAY LIVE2024〜23歳のまきちゃんも1番かわいいよー！〜』

ベルサール虎ノ門
開場16:00  開演16:30 
SS・S完売御礼
Aチケット 2,000円(別途1D)
http://tiget.net/events/348788
10:59まで前売販売中！

✦ː──────────

＜花いろはタイテ＞
15:10-15:50 前物販(まきほのみ)
16:30-16:40 ライブ①
18:05-19:10 ライブ②
(田北香世子 / 高橋きら / りおりん コラボあり)
19:20-21:20 終演後物販

✦ː──────────

前物販ではチェキ券販売・Tシャツ引取のみ行います。
グッズ販売に関しましては開場時〜とさせていただきます。
その他レギュレーション・注意事項はPOPの方ご確認お願いいたします。

まきほのお誕生日、一緒に最高の1日にしましょう！
お待ちしております
photo_links: ['https://pbs.twimg.com/media/GaNT2aNbAAA2hav?format=jpg', 'https://pbs.twimg.com/media/GaNT2aPbUAQbyqS?format=jpg']
media_embedded_link: False


In [342]:
#自分にリプ＋引用（画像あり）
account_name = "hanairoha_ww"
tweet_id="1850511705752895507"
get_tweet_info_by_id(account_name, tweet_id)

tweet_text: 11/14(木)
衛星とカラテアPresents
『超 明星現象』
WWWX / WWW 他全8会場予定
2ショット写メor入場1pt＋30秒トーク延長券
 http://t-dv.com/chomyojo_1114
photo_links: []
media_embedded_link: False
q_tweet: 1850510635991470287


In [11]:

driver.find_element(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()


In [ ]:
from lxml import etree

def find_xpath(html_content, element_text):
    # BeautifulSoup で HTML をパース
    soup = BeautifulSoup(html_content, "html.parser")
    
    # BeautifulSoup のオブジェクトを lxml に変換
    dom = etree.HTML(str(soup))
    
    # 部分一致する要素を探索し、XPathを取得
    for elem in dom.xpath(f"//*[contains(text(), '{element_text}')]"):
        # 要素の XPath を取得して表示
        xpath = dom.getroottree().getpath(elem)
        print(f"Element text: {elem.text} - XPath: {xpath}")
        
# 使用例
html_content = '''<div class="css-175oi2r r-18kxxzh r-1wron08 r-onrtq4 r-1awozwy"><div class="css-175oi2r" data-testid="Tweet-User-Avatar"><div class="css-175oi2r r-18kxxzh r-1wbh5a2 r-13qz1uu"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs"><div class="css-175oi2r r-bztko3 r-1adg3ll" data-testid="UserAvatar-Container-hanairoha_ww" style="width: 40px; height: 40px;"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 100%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1adg3ll r-1pi2tsx r-13qz1uu r-45ll9u r-u8s1d r-1v2oles r-176fswd r-bztko3"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 100%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-sdzlij r-1udh08x r-5f1w11 r-u8s1d r-8jfcpp" style="width: calc(100% + 4px); height: calc(100% + 4px);"><a href="/hanairoha_ww" aria-hidden="true" role="link" tabindex="-1" class="css-175oi2r r-1pi2tsx r-13qz1uu r-o7ynqc r-6416eg r-1ny4l3l r-1loqt21" style="background-color: rgba(0, 0, 0, 0);"><div class="css-175oi2r r-sdzlij r-1udh08x r-633pao r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-1pi2tsx r-13qz1uu" style="background-color: rgba(0, 0, 0, 0);"></div></div><div class="css-175oi2r r-sdzlij r-1udh08x r-633pao r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-1pi2tsx r-13qz1uu r-kemksi"></div></div><div class="css-175oi2r r-sdzlij r-1udh08x r-633pao r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="background-color: rgb(0, 0, 0); width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-1adg3ll r-1udh08x" style=""><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 100%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/profile_images/1509732153239937029/3ZKO3fIK_bigger.jpg&quot;);"></div><img alt="" draggable="true" src="https://pbs.twimg.com/profile_images/1509732153239937029/3ZKO3fIK_bigger.jpg" class="css-9pa8cd"></div></div></div></div><div class="css-175oi2r r-sdzlij r-1udh08x r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-172uzmj r-1pi2tsx r-13qz1uu r-o7ynqc r-6416eg r-1ny4l3l"></div></div></a></div></div></div></div></div></div></div></div><div class="css-175oi2r r-1bnu78o r-f8sm7e r-m5arl1 r-16y2uox r-14gqq1x"></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu"><div class="css-175oi2r r-zl2h9q"><div class="css-175oi2r r-k4xj1c r-18u37iz r-1wtj0ep"><div class="css-175oi2r r-1d09ksm r-18u37iz r-1wbh5a2"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1awozwy r-18u37iz" id="id__386cmyolbtb" data-testid="User-Name"><div class="css-175oi2r r-1awozwy r-18u37iz r-1wbh5a2 r-dnmrzs"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs"><a href="/hanairoha_ww" role="link" class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1awozwy r-18u37iz r-1wbh5a2 r-dnmrzs"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-b88u0q r-1awozwy r-6koalj r-1udh08x r-3s2u2q" style="text-overflow: unset; color: rgb(231, 233, 234);"><span class="css-1jxf684 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">【公式】花いろは</span></span></div><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-18u37iz r-1q142lx" style="text-overflow: unset; color: rgb(231, 233, 234);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-1awozwy r-xoduu5" style="text-overflow: unset;"></span></div></div></a></div></div><div class="css-175oi2r r-18u37iz r-1wbh5a2 r-1ez5h0i"><div class="css-175oi2r r-1d09ksm r-18u37iz r-1wbh5a2"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs"><a href="/hanairoha_ww" role="link" tabindex="-1" class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1loqt21"><div dir="ltr" class="css-146c3p1 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-18u37iz r-1wvb978" style="text-overflow: unset; color: rgb(113, 118, 123);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">@hanairoha_ww</span></div></a></div><div dir="ltr" aria-hidden="true" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-1q142lx r-n7gxbd" style="text-overflow: unset; color: rgb(113, 118, 123);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">·</span></div><div class="css-175oi2r r-18u37iz r-1q142lx"><a href="/hanairoha_ww/status/1839241504373813483" dir="ltr" aria-label="9月26日" role="link" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-1q142lx r-1w6e6rj r-9aw3ui r-3s2u2q r-1loqt21" style="text-overflow: unset; color: rgb(113, 118, 123);"><time datetime="2024-09-26T09:52:00.000Z">9月26日</time></a></div></div></div></div></div></div><div class="css-175oi2r r-1kkk96v"><div class="css-175oi2r r-1awozwy r-18u37iz r-1cmwbt1 r-1wtj0ep"><div class="css-175oi2r r-1awozwy r-6koalj r-18u37iz"><div class="css-175oi2r"><div class="css-175oi2r r-18u37iz r-1h0z5md"><button aria-expanded="false" aria-haspopup="menu" aria-label="もっと見る" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="caret" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M3 12c0-1.1.9-2 2-2s2 .9 2 2-.9 2-2 2-2-.9-2-2zm9 2c1.1 0 2-.9 2-2s-.9-2-2-2-2 .9-2 2 .9 2 2 2zm7 0c1.1 0 2-.9 2-2s-.9-2-2-2-2 .9-2 2 .9 2 2 2z"></path></g></svg></div></div></button></div></div></div></div></div></div></div><div class="css-175oi2r"><div dir="auto" lang="ja" class="css-146c3p1 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim" id="id__uxf5eumay2" data-testid="tweetText" style="-webkit-line-clamp: 10; text-overflow: unset; color: rgb(231, 233, 234);"><span class="r-18u37iz"><a dir="ltr" href="/hashtag/花いろは?src=hashtag_click" role="link" class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3 r-1loqt21" style="text-overflow: unset; color: rgb(29, 155, 240);">#花いろは</a></span><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;"> 9/27〜10/3まで7日連続ライブ</span><img alt="📢" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f4e2.svg" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><img alt="⚡️" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/26a1.svg" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">

来場日数が4日間・7日間の方に特典プレゼント決定</span><img alt="🎁" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f381.svg" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">
4pt...2ショット写メ(トークあり)
7pt...3ショット写メor集合写メor10秒動画(全て撮影のみ)

✦ː──────────

9/27(金)
『 GIRLS BOX VOL.245 〜ReNY10周年おめでとうSP!!〜 』
</span><img alt="📍" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f4cd.svg" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">新宿ReNY</span></div><div dir="ltr" tabindex="0" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-1loqt21" data-testid="tweet-text-show-more-link" style="text-overflow: unset; color: rgb(29, 155, 240);" role="link"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">さらに表示</span></div></div><div aria-labelledby="id__25eegn418qk id__izekk86wli" class="css-175oi2r r-9aw3ui r-1s2bzr4" id="id__3fnfswoecko"><div class="css-175oi2r r-9aw3ui"><div class="css-175oi2r"><div class="css-175oi2r"><div class="css-175oi2r r-1kqtdi0 r-1phboty r-rs99b7 r-1867qdf r-1udh08x r-o7ynqc r-6416eg r-1ny4l3l"><div class="css-175oi2r"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1839241504373813483/photo/1" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1adg3ll r-1udh08x"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 70.7031%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GYZMZSFWUAAiTqj?format=jpg&amp;name=900x900&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GYZMZSFWUAAiTqj?format=jpg&amp;name=900x900" class="css-9pa8cd"></div></div></div></a></div></div></div></div></div></div></div><div class="css-175oi2r"><div class="css-175oi2r"><div aria-label="1 件の返信、11 件のリポスト、33 件のいいね、25765 件の表示" role="group" class="css-175oi2r r-1kbdv8c r-18u37iz r-1wtj0ep r-1ye8kvj r-1s2bzr4" id="id__ovolpwrtj2q"><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><button aria-label="1 件の返信。返信する" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="reply" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M1.751 10c0-4.42 3.584-8 8.005-8h4.366c4.49 0 8.129 3.64 8.129 8.13 0 2.96-1.607 5.68-4.196 7.11l-8.054 4.46v-3.69h-.067c-4.49.1-8.183-3.51-8.183-8.01zm8.005-6c-3.317 0-6.005 2.69-6.005 6 0 3.37 2.77 6.08 6.138 6.01l.351-.01h1.761v2.3l5.087-2.81c1.951-1.08 3.163-3.13 3.163-5.36 0-3.39-2.744-6.13-6.129-6.13H9.756z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">1</span></span></span></div></div></button></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><button aria-expanded="false" aria-haspopup="menu" aria-label="11 件のリポスト件。リポスト" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="retweet" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M4.5 3.88l4.432 4.14-1.364 1.46L5.5 7.55V16c0 1.1.896 2 2 2H13v2H7.5c-2.209 0-4-1.79-4-4V7.55L1.432 9.48.068 8.02 4.5 3.88zM16.5 6H11V4h5.5c2.209 0 4 1.79 4 4v8.45l2.068-1.93 1.364 1.46-4.432 4.14-4.432-4.14 1.364-1.46 2.068 1.93V8c0-1.1-.896-2-2-2z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">11</span></span></span></div></div></button></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><button aria-label="33 件のいいね。いいねする" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="like" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M16.697 5.5c-1.222-.06-2.679.51-3.89 2.16l-.805 1.09-.806-1.09C9.984 6.01 8.526 5.44 7.304 5.5c-1.243.07-2.349.78-2.91 1.91-.552 1.12-.633 2.78.479 4.82 1.074 1.97 3.257 4.27 7.129 6.61 3.87-2.34 6.052-4.64 7.126-6.61 1.111-2.04 1.03-3.7.477-4.82-.561-1.13-1.666-1.84-2.908-1.91zm4.187 7.69c-1.351 2.48-4.001 5.12-8.379 7.67l-.503.3-.504-.3c-4.379-2.55-7.029-5.19-8.382-7.67-1.36-2.5-1.41-4.86-.514-6.67.887-1.79 2.647-2.91 4.601-3.01 1.651-.09 3.368.56 4.798 2.01 1.429-1.45 3.146-2.1 4.796-2.01 1.954.1 3.714 1.22 4.601 3.01.896 1.81.846 4.17-.514 6.67z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">33</span></span></span></div></div></button></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><a href="/hanairoha_ww/status/1839241504373813483/analytics" aria-label="25765 件の表示。ポストアナリティクスを表示" role="link" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1ny4l3l r-1loqt21"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M8.75 21V3h2v18h-2zM18 21V8.5h2V21h-2zM4 21l.004-10h2L6 21H4zm9.248 0v-7h2v7h-2z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">2.5万</span></span></span></div></div></a></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-1wron08"><button aria-label="ブックマーク" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="bookmark" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M4 4.5C4 3.12 5.119 2 6.5 2h11C18.881 2 20 3.12 20 4.5v18.44l-8-5.71-8 5.71V4.5zM6.5 4c-.276 0-.5.22-.5.5v14.56l6-4.29 6 4.29V4.5c0-.28-.224-.5-.5-.5h-11z"></path></g></svg></div></div></button></div><div class="css-175oi2r" style="justify-content: inherit; display: inline-grid; transform: rotate(0deg) scale(1) translate3d(0px, 0px, 0px);"><div class="css-175oi2r r-18u37iz r-1h0z5md"><button aria-expanded="false" aria-haspopup="menu" aria-label="ポストを共有" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M12 2.59l5.7 5.7-1.41 1.42L13 6.41V16h-2V6.41l-3.3 3.3-1.41-1.42L12 2.59zM21 15l-.02 3.51c0 1.38-1.12 2.49-2.5 2.49H5.5C4.11 21 3 19.88 3 18.5V15h2v3.5c0 .28.22.5.5.5h12.98c.28 0 .5-.22.5-.5L19 15h2z"></path></g></svg></div></div></button></div></div></div></div></div></div>
'''

# 特定の要素に部分一致するテキストを指定
find_xpath(html_content, "9月26日")


Element text: 9月26日 - XPath: /html/body/div[2]/div[1]/div/div[1]/div/div/div[2]/div/div[3]/a/time


In [ ]:
#画像4つ
html_content = '''<div class="css-175oi2r"><div class="css-175oi2r"><div class="css-175oi2r r-1kqtdi0 r-1phboty r-rs99b7 r-1867qdf r-1udh08x r-o7ynqc r-6416eg r-1ny4l3l"><div class="css-175oi2r r-1adg3ll r-1udh08x"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 56.25%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1pi2tsx r-13qz1uu r-eqz5dr"><div class="css-175oi2r r-zl2h9q r-1iusvr4 r-16y2uox r-18u37iz"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim r-x1x4zq"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1833422444323013007/photo/1" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GXGf_pZbwAA_W7f?format=jpg&amp;name=360x360&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GXGf_pZbwAA_W7f?format=jpg&amp;name=360x360" class="css-9pa8cd"></div></div></a></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1833422444323013007/photo/2" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GXGf_pYasAAUJU0?format=jpg&amp;name=360x360&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GXGf_pYasAAUJU0?format=jpg&amp;name=360x360" class="css-9pa8cd"></div></div></a></div></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-18u37iz"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim r-x1x4zq"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1833422444323013007/photo/3" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GXGf_pUbEAEJtD0?format=jpg&amp;name=360x360&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GXGf_pUbEAEJtD0?format=jpg&amp;name=360x360" class="css-9pa8cd"></div></div></a></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1833422444323013007/photo/4" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GXGf_pWbEAI5_K9?format=jpg&amp;name=360x360&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GXGf_pWbEAI5_K9?format=jpg&amp;name=360x360" class="css-9pa8cd"></div></div></a></div></div></div></div></div></div></div></div></div>
'''
print(extract_tag_structure(html_content))

<div>
 <div>
  <div>
   <div>
    <div>
    </div>
    <div>
     <div>
      <div>
       <div>
        <div>
         <a>
          <div>
           <div>
            <div>
            </div>
            <img/>
           </div>
          </div>
         </a>
        </div>
       </div>
       <div>
        <div>
         <a>
          <div>
           <div>
            <div>
            </div>
            <img/>
           </div>
          </div>
         </a>
        </div>
       </div>
      </div>
      <div>
       <div>
        <div>
         <a>
          <div>
           <div>
            <div>
            </div>
            <img/>
           </div>
          </div>
         </a>
        </div>
       </div>
       <div>
        <div>
         <a>
          <div>
           <div>
            <div>
            </div>
            <img/>
           </div>
          </div>
         </a>
        </div>
       </div>
      </div>
     </div>
    </div>
   </div>
  </div>
 </div>


In [ ]:
#画像・動画混在
html_content = '''<div class="css-175oi2r"><div class="css-175oi2r"><div class="css-175oi2r r-1kqtdi0 r-1phboty r-rs99b7 r-1867qdf r-1udh08x r-o7ynqc r-6416eg r-1ny4l3l"><div class="css-175oi2r r-1adg3ll r-1udh08x"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 56.25%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1pi2tsx r-13qz1uu r-18u37iz"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim r-x1x4zq"><div class="css-175oi2r r-1pi2tsx" data-testid="tweetPhoto"><div class="css-175oi2r" style=""><div class="css-175oi2r r-1adg3ll r-1udh08x"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 112.811%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="placementTracking"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="videoPlayer"><div class="css-175oi2r r-1adg3ll r-1udh08x r-bnwqim r-1pi2tsx r-13qz1uu"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 112.811%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div data-testid="videoComponent" style="height: 100%; position: relative; transform: translateZ(0px); width: 100%;"><div style="height: 100%; position: absolute; width: 100%;"><div style="position: relative; width: 100%; height: 100%; background-color: transparent; overflow: hidden;"><video preload="none" tabindex="-1" playsinline="" aria-label="埋め込み動画" poster="https://pbs.twimg.com/amplify_video_thumb/1816100909442912256/img/j2gIn4x_JUuy_1cS.jpg" style="width: 100%; height: 100%; position: absolute; background-color: black; top: 0%; left: 0%; transform: rotate(0deg) scale(1.005);"><source type="video/mp4" src="blob:https://x.com/cb29d7f6-22ff-4a62-9448-8f6ff77d7be0"></video></div></div><div tabindex="0" class="css-175oi2r r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af"><div class="css-175oi2r r-1p0dtai r-1loqt21 r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="video-player-mini-ui-"></div><button aria-label="動画を再生" role="button" class="css-175oi2r r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1awozwy r-1777fci r-1loqt21 r-1ny4l3l" type="button"><div class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/amplify_video_thumb/1816100909442912256/img/j2gIn4x_JUuy_1cS.jpg&quot;);"></div><img alt="" draggable="false" src="https://pbs.twimg.com/amplify_video_thumb/1816100909442912256/img/j2gIn4x_JUuy_1cS.jpg" class="css-9pa8cd"></div></button><div class="css-175oi2r r-18u37iz r-vampta r-u8s1d r-rki7wi r-1f55h46 r-633pao"><div class="css-175oi2r r-1awozwy r-k200y r-z2wwpe r-z80fyv r-1777fci r-loe9s5 r-18u37iz r-1mkv55d r-1e084wi r-108j81s r-1nna3df r-yc4bad r-ne48ov r-633pao"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-qvutc0 r-1tl8opc r-q4m81j r-n6v787 r-1cwl3u0 r-16dba41 r-lrvibr" style="color: rgb(255, 255, 255); text-overflow: unset;"><div class="css-175oi2r r-1awozwy r-6koalj r-18u37iz"><button aria-label="一時停止" role="button" class="css-175oi2r r-xoduu5 r-tuq35u r-105ug2t r-1loqt21 r-o7ynqc r-6416eg r-1ny4l3l" type="button"><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-tbmifm r-16eto9q"><g><path d="M4 2h5v20H4V2zm11 20h5V2h-5v20z"></path></g></svg></button></div></div></div></div><div style="transition-duration: 250ms; transition-property: opacity, height; transition-timing-function: ease; will-change: opacity; opacity: 0;"></div></div></div></div></div></div></div></div></div></div></div></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1816101043849290094/photo/2" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GTQWJ60aYAMaWOZ?format=jpg&amp;name=small&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GTQWJ60aYAMaWOZ?format=jpg&amp;name=small" class="css-9pa8cd"></div></div></a></div></div></div></div></div></div></div></div>'''
print(extract_tag_structure(html_content))

<div>
 <div>
  <div>
   <div>
    <div>
    </div>
    <div>
     <div>
      <div>
       <div>
        <div>
         <div>
          <div>
          </div>
          <div>
           <div>
            <div>
             <div>
              <div>
              </div>
              <div>
               <div>
                <div>
                 <div>
                  <video>
                   <source/>
                  </video>
                 </div>
                </div>
                <div>
                 <div>
                  <div>
                  </div>
                  <button>
                   <div>
                    <div>
                    </div>
                    <img/>
                   </div>
                  </button>
                  <div>
                   <div>
                    <div>
                     <div>
                      <button>
                       <svg>
                        <g>
                         <path>
             

In [ ]:
html_content = """<div class="css-175oi2r r-eqz5dr r-16y2uox r-1wbh5a2"><div class="css-175oi2r r-16y2uox r-1wbh5a2 r-1ny4l3l"><div class="css-175oi2r"><div class="css-175oi2r r-18u37iz"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-ttdzmv"></div></div></div><div class="css-175oi2r r-18u37iz"><div class="css-175oi2r r-18kxxzh r-1wron08 r-onrtq4 r-1awozwy"><div class="css-175oi2r" data-testid="Tweet-User-Avatar"><div class="css-175oi2r r-18kxxzh r-1wbh5a2 r-13qz1uu"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs"><div class="css-175oi2r r-bztko3 r-1adg3ll" data-testid="UserAvatar-Container-hanairoha_ww" style="width: 40px; height: 40px;"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 100%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1adg3ll r-1pi2tsx r-13qz1uu r-45ll9u r-u8s1d r-1v2oles r-176fswd r-bztko3"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 100%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-sdzlij r-1udh08x r-5f1w11 r-u8s1d r-8jfcpp" style="width: calc(100% + 4px); height: calc(100% + 4px);"><a href="/hanairoha_ww" aria-hidden="true" role="link" tabindex="-1" class="css-175oi2r r-1pi2tsx r-13qz1uu r-o7ynqc r-6416eg r-1ny4l3l r-1loqt21" style="background-color: rgba(0, 0, 0, 0);"><div class="css-175oi2r r-sdzlij r-1udh08x r-633pao r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-1pi2tsx r-13qz1uu" style="background-color: rgba(0, 0, 0, 0);"></div></div><div class="css-175oi2r r-sdzlij r-1udh08x r-633pao r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-1pi2tsx r-13qz1uu r-kemksi"></div></div><div class="css-175oi2r r-sdzlij r-1udh08x r-633pao r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="background-color: rgb(0, 0, 0); width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-1adg3ll r-1udh08x" style=""><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 100%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/profile_images/1509732153239937029/3ZKO3fIK_normal.jpg&quot;);"></div><img alt="" draggable="true" src="https://pbs.twimg.com/profile_images/1509732153239937029/3ZKO3fIK_normal.jpg" class="css-9pa8cd"></div></div></div></div><div class="css-175oi2r r-sdzlij r-1udh08x r-45ll9u r-u8s1d r-1v2oles r-176fswd" style="width: calc(100% - 4px); height: calc(100% - 4px);"><div class="css-175oi2r r-172uzmj r-1pi2tsx r-13qz1uu r-o7ynqc r-6416eg r-1ny4l3l"></div></div></a></div></div></div></div></div></div></div></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu"><div class="css-175oi2r r-zl2h9q"><div class="css-175oi2r r-k4xj1c r-18u37iz r-1wtj0ep"><div class="css-175oi2r r-1d09ksm r-18u37iz r-1wbh5a2"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1awozwy r-18u37iz" id="id__hxxqrwsr2ao" data-testid="User-Name"><div class="css-175oi2r r-1awozwy r-18u37iz r-1wbh5a2 r-dnmrzs"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs"><a href="/hanairoha_ww" role="link" class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1awozwy r-18u37iz r-1wbh5a2 r-dnmrzs"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-b88u0q r-1awozwy r-6koalj r-1udh08x r-3s2u2q" style="text-overflow: unset; color: rgb(231, 233, 234);"><span class="css-1jxf684 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">【公式】花いろは</span></span></div><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-18u37iz r-1q142lx" style="text-overflow: unset; color: rgb(231, 233, 234);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-1awozwy r-xoduu5" style="text-overflow: unset;"></span></div></div></a></div></div><div class="css-175oi2r r-18u37iz r-1wbh5a2 r-1ez5h0i"><div class="css-175oi2r r-1d09ksm r-18u37iz r-1wbh5a2"><div class="css-175oi2r r-1wbh5a2 r-dnmrzs"><a href="/hanairoha_ww" role="link" tabindex="-1" class="css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1loqt21"><div dir="ltr" class="css-146c3p1 r-dnmrzs r-1udh08x r-3s2u2q r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-18u37iz r-1wvb978" style="text-overflow: unset; color: rgb(113, 118, 123);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">@hanairoha_ww</span></div></a></div><div dir="ltr" aria-hidden="true" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-1q142lx r-n7gxbd" style="text-overflow: unset; color: rgb(113, 118, 123);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">·</span></div><div class="css-175oi2r r-18u37iz r-1q142lx"><div class="css-175oi2r r-1d09ksm r-18u37iz r-1wbh5a2"><a href="/hanairoha_ww/status/1852332917802045648" dir="ltr" aria-label="11月1日" role="link" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-1q142lx r-1w6e6rj r-9aw3ui r-3s2u2q r-1loqt21" style="text-overflow: unset; color: rgb(113, 118, 123);"><time datetime="2024-11-01T12:52:36.000Z">11月1日</time></a></div></div></div></div></div></div></div><div class="css-175oi2r r-1kkk96v"><div class="css-175oi2r r-1awozwy r-18u37iz r-1cmwbt1 r-1wtj0ep"><div class="css-175oi2r r-1awozwy r-6koalj r-18u37iz"><div class="css-175oi2r"><div class="css-175oi2r r-18u37iz r-1h0z5md"><button aria-expanded="false" aria-haspopup="menu" aria-label="もっと見る" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="caret" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M3 12c0-1.1.9-2 2-2s2 .9 2 2-.9 2-2 2-2-.9-2-2zm9 2c1.1 0 2-.9 2-2s-.9-2-2-2-2 .9-2 2 .9 2 2 2zm7 0c1.1 0 2-.9 2-2s-.9-2-2-2-2 .9-2 2 .9 2 2 2z"></path></g></svg></div></div></button></div></div></div></div></div></div></div><div class="css-175oi2r"><div dir="auto" lang="ja" class="css-146c3p1 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim" id="id__mdggot2el9g" data-testid="tweetText" style="-webkit-line-clamp: 10; text-overflow: unset; color: rgb(231, 233, 234);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">明日のライブ情報</span><img alt="📢" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f4e2.svg" title="拡声器" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><img alt="🌟" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f31f.svg" title="光る星" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">

˗ˏˋ  関西ツアー初日</span><img alt="❕" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/2755.svg" title="白いビックリマーク" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">  ˎˊ˗

11/2(土)
関西ツアー2024『いろはにほへと』
</span><img alt="📍" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f4cd.svg" title="画鋲" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">＜滋賀＞滋賀U★STONE

</span><img alt="⏰" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/23f0.svg" title="目覚まし時計" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">OPEN17:00 / START17:30
</span><img alt="💰" draggable="false" src="https://abs-0.twimg.com/emoji/v2/svg/1f4b0.svg" title="ドル袋" class="r-4qtqp9 r-dflpy8 r-k4bwe5 r-1kpi4qh r-pp5qcn r-h9hxbl"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">SSチケット(最前席)6,000円　
Sチケット(前方席)3,000円　
Aチケット(一般エリア)2,000円　 
Bチケット(女性)無料　※各1D別</span></div><div dir="ltr" tabindex="0" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc r-a023e6 r-rjixqe r-16dba41 r-1loqt21" data-testid="tweet-text-show-more-link" style="text-overflow: unset; color: rgb(29, 155, 240);" role="link"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">さらに表示</span></div></div><div aria-labelledby="id__fmw4j295uf6 id__qdejlkflsy" class="css-175oi2r r-9aw3ui r-1s2bzr4" id="id__y3dr02qzwm7"><div class="css-175oi2r r-9aw3ui"><div class="css-175oi2r"><div class="css-175oi2r"><div class="css-175oi2r r-1kqtdi0 r-1phboty r-rs99b7 r-1867qdf r-1udh08x r-o7ynqc r-6416eg r-1ny4l3l"><div class="css-175oi2r r-1adg3ll r-1udh08x"><div class="r-1adg3ll r-13qz1uu" style="padding-bottom: 56.25%;"></div><div class="r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-ipm5af r-13qz1uu"><div class="css-175oi2r r-1pi2tsx r-13qz1uu r-18u37iz"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim r-x1x4zq"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1852332917802045648/photo/1" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GbTM2N9a4AA2UVW?format=jpg&amp;name=360x360&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GbTM2N9a4AA2UVW?format=jpg&amp;name=360x360" class="css-9pa8cd"></div></div></a></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-eqz5dr"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim r-zl2h9q"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1852332917802045648/photo/2" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GbTO-BoboAA-6_b?format=jpg&amp;name=360x360&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GbTO-BoboAA-6_b?format=jpg&amp;name=360x360" class="css-9pa8cd"></div></div></a></div></div><div class="css-175oi2r r-1iusvr4 r-16y2uox r-bnwqim"><div class="css-175oi2r r-16y2uox r-1pi2tsx r-13qz1uu"><a href="/hanairoha_ww/status/1852332917802045648/photo/3" role="link" class="css-175oi2r r-1pi2tsx r-1ny4l3l r-1loqt21"><div class="css-175oi2r r-1p0dtai r-1d2f490 r-1udh08x r-u8s1d r-zchlnj r-ipm5af"><div aria-label="画像" class="css-175oi2r r-1mlwlqe r-1udh08x r-417010 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af" data-testid="tweetPhoto" style="margin: 0px;"><div class="css-175oi2r r-1niwhzg r-vvn4in r-u6sd8q r-1p0dtai r-1pi2tsx r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-13qz1uu r-1wyyakw r-4gszlv" style="background-image: url(&quot;https://pbs.twimg.com/media/GbTO-BraQAAT2eU?format=jpg&amp;name=small&quot;);"></div><img alt="画像" draggable="true" src="https://pbs.twimg.com/media/GbTO-BraQAAT2eU?format=jpg&amp;name=small" class="css-9pa8cd"></div></div></a></div></div></div></div></div></div></div></div></div></div></div><div class="css-175oi2r"><div class="css-175oi2r"><div aria-label="8 件のリポスト、31 件のいいね、8604 件の表示" role="group" class="css-175oi2r r-1kbdv8c r-18u37iz r-1wtj0ep r-1ye8kvj r-1s2bzr4" id="id__5ra8msskgjf"><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><button aria-label="0 件の返信。返信する" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="reply" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M1.751 10c0-4.42 3.584-8 8.005-8h4.366c4.49 0 8.129 3.64 8.129 8.13 0 2.96-1.607 5.68-4.196 7.11l-8.054 4.46v-3.69h-.067c-4.49.1-8.183-3.51-8.183-8.01zm8.005-6c-3.317 0-6.005 2.69-6.005 6 0 3.37 2.77 6.08 6.138 6.01l.351-.01h1.761v2.3l5.087-2.81c1.951-1.08 3.163-3.13 3.163-5.36 0-3.39-2.744-6.13-6.129-6.13H9.756z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"></span></span></div></div></button></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><button aria-expanded="false" aria-haspopup="menu" aria-label="8 件のリポスト件。リポスト" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="retweet" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M4.5 3.88l4.432 4.14-1.364 1.46L5.5 7.55V16c0 1.1.896 2 2 2H13v2H7.5c-2.209 0-4-1.79-4-4V7.55L1.432 9.48.068 8.02 4.5 3.88zM16.5 6H11V4h5.5c2.209 0 4 1.79 4 4v8.45l2.068-1.93 1.364 1.46-4.432 4.14-4.432-4.14 1.364-1.46 2.068 1.93V8c0-1.1-.896-2-2-2z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">8</span></span></span></div></div></button></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><button aria-label="31 件のいいね。いいねする" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="like" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M16.697 5.5c-1.222-.06-2.679.51-3.89 2.16l-.805 1.09-.806-1.09C9.984 6.01 8.526 5.44 7.304 5.5c-1.243.07-2.349.78-2.91 1.91-.552 1.12-.633 2.78.479 4.82 1.074 1.97 3.257 4.27 7.129 6.61 3.87-2.34 6.052-4.64 7.126-6.61 1.111-2.04 1.03-3.7.477-4.82-.561-1.13-1.666-1.84-2.908-1.91zm4.187 7.69c-1.351 2.48-4.001 5.12-8.379 7.67l-.503.3-.504-.3c-4.379-2.55-7.029-5.19-8.382-7.67-1.36-2.5-1.41-4.86-.514-6.67.887-1.79 2.647-2.91 4.601-3.01 1.651-.09 3.368.56 4.798 2.01 1.429-1.45 3.146-2.1 4.796-2.01 1.954.1 3.714 1.22 4.601 3.01.896 1.81.846 4.17-.514 6.67z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">31</span></span></span></div></div></button></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"><a href="/hanairoha_ww/status/1852332917802045648/analytics" aria-label="8604 件の表示。ポストアナリティクスを表示" role="link" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1ny4l3l r-1loqt21"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M8.75 21V3h2v18h-2zM18 21V8.5h2V21h-2zM4 21l.004-10h2L6 21H4zm9.248 0v-7h2v7h-2z"></path></g></svg></div><div class="css-175oi2r r-xoduu5 r-1udh08x"><span data-testid="app-text-transition-container" style="transition-property: transform; transition-duration: 0.3s; transform: translate3d(0px, 0px, 0px);"><span class="css-1jxf684 r-1ttztb7 r-qvutc0 r-1tl8opc r-n6v787 r-1cwl3u0 r-1k6nrdp r-n7gxbd" style="text-overflow: unset;"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-1tl8opc" style="text-overflow: unset;">8,604</span></span></span></div></div></a></div><div class="css-175oi2r r-18u37iz r-1h0z5md r-1wron08"><button aria-label="ブックマーク" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" data-testid="bookmark" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M4 4.5C4 3.12 5.119 2 6.5 2h11C18.881 2 20 3.12 20 4.5v18.44l-8-5.71-8 5.71V4.5zM6.5 4c-.276 0-.5.22-.5.5v14.56l6-4.29 6 4.29V4.5c0-.28-.224-.5-.5-.5h-11z"></path></g></svg></div></div></button></div><div class="css-175oi2r" style="justify-content: inherit; display: inline-grid; transform: rotate(0deg) scale(1) translate3d(0px, 0px, 0px);"><div class="css-175oi2r r-18u37iz r-1h0z5md"><button aria-expanded="false" aria-haspopup="menu" aria-label="ポストを共有" role="button" class="css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l" type="button"><div dir="ltr" class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-1awozwy r-6koalj r-1h0z5md r-o7ynqc r-clp7b1 r-3s2u2q" style="text-overflow: unset; color: rgb(113, 118, 123);"><div class="css-175oi2r r-xoduu5"><div class="css-175oi2r r-xoduu5 r-1p0dtai r-1d2f490 r-u8s1d r-zchlnj r-ipm5af r-1niwhzg r-sdzlij r-xf4iuw r-o7ynqc r-6416eg r-1ny4l3l"></div><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1xvli5t r-1hdv0qi"><g><path d="M12 2.59l5.7 5.7-1.41 1.42L13 6.41V16h-2V6.41l-3.3 3.3-1.41-1.42L12 2.59zM21 15l-.02 3.51c0 1.38-1.12 2.49-2.5 2.49H5.5C4.11 21 3 19.88 3 18.5V15h2v3.5c0 .28.22.5.5.5h12.98c.28 0 .5-.22.5-.5L19 15h2z"></path></g></svg></div></div></button></div></div></div></div></div></div></div></div></div>"""
print(extract_tag_structure(html_content))



article/
div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]/div/div[3]/a

article/
div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]/div/div[3]/div/a

div/div/div[2]/div[2]/div[1]/div/div/div[1]/div/div[2]/div/div[3]/a

<div>
 <div>
  <div>
   <div>
    <div>
    </div>
   </div>
  </div>
  <div>
   <div>
    <div>
     <div>
      <div>
       <div>
        <div>
        </div>
        <div>
         <div>
          <div>
          </div>
          <div>
           <div>
            <a>
             <div>
              <div>
              </div>
             </div>
             <div>
              <div>
              </div>
             </div>
             <div>
              <div>
               <div>
               </div>
               <div>
                <div>
                 <div>
                 </div>
                 <img/>
                </div>
               </div>
              </div>
             </div>
             <div>
              <div>
              </div>
             </div>
            </a>
           </div>
          </div>
         </div>
        </div>
       </div>
      </div>
     </div>
    </div>
   </div>
   <div>
    <div>
     <div>
      <div>
       <div>
       

In [381]:
def get_tweet_info_simple(tweet):
    # tweet_text = []

    #固定ツイートか否かの判定がしたい
    for i,element_tweet in enumerate(tweet.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")):
        if i == 0:#ツイートのアカウントや日時情報
            try:
                account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
                tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
            except NoSuchElementException:
                try:#??????
                    account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
                    tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/div/a').get_attribute("href").split("/")[-1]
                except NoSuchElementException:
                    print(tweet.get_attribute("innerHTML"))
        # tweet_text.append(element_tweet.text)
    return {"account_name":account_name,"tweet_id":tweet_id}#,"tweet_text":tweet_text,"html":tweet.get_attribute("innerHTML")}

In [1]:
#ツイートオブジェクトから必要な情報を抜き出して構造化する関数
# def get_info_from_tweet(tweet,target_account_name):
#     for i,element_tweet in enumerate(tweet.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")):
#     # print(i)
#     # print(element_tweet.get_attribute("innerHTML"))
#         if i == 0:#ツイートのアカウントや日時情報
#             account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
#             if account_name != target_account_name: #プロモツイートを排除
#                 return False
#             time_str = dt.datetime.strftime(
#                 dt.datetime.strptime(
#                 element_tweet.find_element(By.TAG_NAME,'time').get_attribute("datetime")
#                 ,'%Y-%m-%dT%H:%M:%S.%fZ')
#                     + dt.timedelta(seconds=60*60*9),'%Y%m%d%H%M%S')
#             tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
#             # print(time_str)
#             # print(tweet_id)
#         if i == 2:
#             tweet_text = element_tweet.text
#             # print(tweet_text)
#         if i == 3:
#             #一ツイートに複数画像があっても構造はほぼ一緒
#             photo_url = []
#             for j,photo_link in enumerate(element_tweet.find_elements(By.TAG_NAME,'img')):
#                 photo_url.append(photo_link.get_attribute("src"))
#     return {"time_str":time_str,"tweet_id":tweet_id,"tweet_text":tweet_text,"photo_url":photo_url,"html":tweet.get_attribute("innerHTML")}


In [11]:
#アカウント名とTweetIDからその中身のデータを取得する関数
# def get_info_from_tweet_by_tweetid(account_name,tweet_id):
account_name = "hanairoha_ww"
tweet_id="1845006356631388450"
driver.get("https://x.com/{account_name}/status/{tweet_id}".format(account_name=account_name,tweet_id=tweet_id))

In [16]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
formatted_html = soup.prettify()
print(formatted_html)

<html dir="ltr" lang="ja" style="overflow-y: scroll; overscroll-behavior-y: none; font-size: 15px; color-scheme: dark;">
 <head>
  <style id="draftjs-styles_0">
   .draftjs-styles_0 .public-DraftEditorPlaceholder-root {
      color: #71767B;
    }

    .draftjs-styles_0 .public-DraftEditorPlaceholder-hasFocus {
      color: #71767B;
    }
  </style>
  <style id="draftjs-styles">
   .DraftEditor-editorContainer, .DraftEditor-root, .public-DraftEditor-content {
        height: inherit;
        max-height: inherit;
        min-height: inherit;
        text-align: initial;
    }

    .public-DraftEditor-content[contenteditable=true] {
        -moz-user-modify: read-write-plaintext-only;
        -webkit-user-modify: read-write-plaintext-only;
    }

    .format-inline .public-DraftEditor-content[contenteditable=true] {
      -moz-user-modify: read-write;
      -webkit-user-modify: read-write;
    }

    .DraftEditor-root {
        width: 100%;
        position: relative;
        -webkit-use

In [386]:
#TOPからn件のツイートを取得する関数
def get_new_tweets(account_name, n=5):#固定ツイートを除く処理を分岐で入れる？
    #ブラウザを開いてTwitter開いてログインする処理は一旦スキップ
    global driver
    driver.get("https://x.com/{account_name}".format(account_name=account_name))
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "article")))
    tweets={}
    while True:
        for tweet in driver.find_element(By.XPATH,'//section/div/div/div/div').find_elements(By.TAG_NAME,"article"):
            #タイムライン上だと長文ツイートの全体が表示されない（「さらに表示」となる）ので、tweetIDを取得して改めて保存し直す必要あり
            tweet_info = get_tweet_info_simple(tweet)
            if tweet_info:
                if tweet_info["account_name"]==account_name:
                    if tweet_info["tweet_id"] not in tweets:
                        tweets[tweet_info["tweet_id"]]=tweet_info#.get_attribute("innerHTML")#.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")
        if len(tweets)>=n:
            break
        driver.execute_script('window.scrollBy(0, 300);')#あまり大きくするとツイートを飛ばしてしまう
        time.sleep(0.3)
    return tweets

In [387]:
account_name="hanairoha_ww"
driver = twitter_login()
tweets = get_new_tweets(account_name,10)

In [388]:
tweets

{'1838533887288541680': {'account_name': 'hanairoha_ww',
  'tweet_id': '1838533887288541680'},
 '1852866856765518299': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852866856765518299'},
 '1852680625754710151': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852680625754710151'},
 '1852679831185768952': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852679831185768952'},
 '1852525501367333183': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852525501367333183'},
 '1852521730956554436': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852521730956554436'},
 '1852334783222722674': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852334783222722674'},
 '1852332917802045648': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852332917802045648'},
 '1852320784397881456': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852320784397881456'},
 '1852216286962683919': {'account_name': 'hanairoha_ww',
  'tweet_id': '1852216286962683919'}}

In [389]:
for tweet_id in tweets.keys():
    tweets[tweet_id]=get_tweet_info_by_id(account_name, tweet_id)

tweet_time: 20240924200011
tweet_text: 【#花いろは 関西ツアー2024「いろはにほへと」 】

関西ツアーチケット詳細＆ゲスト発表

✦ː──────────

11/2(土)
＜滋賀＞滋賀U★STONE
OPEN17:00 / START17:30
https://tiget.net/events/347147
9/26(木)20:00〜順次

11/3(日)
＜京都＞京都MUSE 
OPEN17:00 / START17:30
ゲスト：なみだ色の消しゴム
 https://tiget.net/events/347359
9/26(木)21:00〜順次

11/4(月祝)
＜兵庫＞神戸VARIT 
OPNE11:30 / START12:00
ゲスト：KRD8
 https://tiget.net/events/347360
9/28(土)20:00〜順次

11/16(土)FINAL
＜大阪＞江坂MUSE 
OPEN18:00 / START18:30
ゲスト：サクヤコノハナ
 https://tiget.net/events/347361
9/28(土)21:00〜順次

✦ː──────────

関西・関東ツアー来場回数特典を予定しております！
ぜひゲストさんのファンの方も一緒に関西ツアー盛り上がっていきましょう
お待ちしております
photo_links: ['https://pbs.twimg.com/media/GYPI0IhakAMLLX5?format=jpg']
media_embedded_link: False
tweet_time: 20241103091417
tweet_text: 本日11/3(日)の #花いろは は

京都公演 ひまり凱旋
✦ː──────────

1本目
『 GIRLS MUSIC SQUARE 
@KYOTO
 』
KYOTO MUSE
開場11:30　開演12:00
当日券
Sチケット(前方) 4,000円 若干枚数
Aチケット(一般) 1,500円 (別途D代必須)
2ショット写メor入場2pt

13:05-13:30
14:00-15:00(終演後)
┈ ┈ ┈ ┈ ┈ 

2本目
関西ツアー『いろはにほへと』
＜京都＞京都MUSE 
OPEN17:00 / S

### ツイート内容が何を示しているのかを判定、構造化
- 出力構造
{
    ""
}

In [391]:

from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    OpenAI.api_key = openai_api_key
client = OpenAI()

GPT_MODEL_NAME = "gpt-4o"

In [392]:
prompt_system = """あなたはライブ情報をキュレーションして公開する会社の優秀な社員です。
あなたの役割はライブの情報を収集しデータ化することです。
ライブの情報はSNSのテキストデータから探し出します。

ユーザーから与えられるテキストデータは様々な内容であり、
必ずしもライブの情報が含まれているとは限りません。
まずはライブ情報が含まれているかを判定し、
含まれている場合にはその内容を出力してください。
ライブの情報が含まれていない場合には、端的に「ライブ情報ではない」と出力してください。

"""

In [393]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_live_info", #関数の名前
            "description": "ライブの情報を出力する", #関数の説明
            "parameters": {
                "type": "object",
                "properties": { #関数の引数の説明
                    # "live_info_flag": {
                    #     "type": "bool",
                    #     "description": "ライブやイベントの情報を表しているかどうかのフラグ",
                    #     "enum": [True, False],
                    # },
                    "live_info": {
                        "type": "object",#"list"
                        "description": "ライブやイベントの情報。複数の異なるイベントの情報が含まれている場合は別個に出力する。同じイベントが複数日に渡って行われる場合は一つのオブジェクトでよい。",
                        "properties": {
                            "live_name": {
                                "type": "string",
                                "description": "ライブイベント名。例）TOKYO IDOL FESTIVAL2024、SUPER☆GiRLSノンストップライブ東京公演",
                            },
                            "live_place": {
                                "type": "string",
                                "description": "ライブイベントの会場名。例）Zepp Tokyo、お台場エリア",
                            },
                            "live_date": {
                                "type": "string",
                                "description": "ライブイベントの行われる日付。複数日行われる場合は開始日を記載。フォーマットはYYYY/MM/DD",
                            },
                        }
                    },
                },
                "required": ["live_info"],
            },
        },
    }
]

In [394]:
def get_live_info(prompt_user):
    response = client.chat.completions.create(
    # response = await client_async.chat.completions.create(
        model=GPT_MODEL_NAME,
        messages=[
            {
                "role": "system",
                "content": prompt_system
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_user},
                    # {
                    #     "type": "image_url",
                    #     "image_url": {
                    #         "url": f"data:image/jpeg;base64,{base64_image}",
                    #     },
                    # },
                ],
            }
        ],
        tools=tools,
    )
    return response

In [405]:
for tweet_id in tweets.keys():
    tweet_text = tweets[tweet_id]["tweet_text"]
    response = get_live_info(tweet_text)
    tweets[tweet_id]["gpt_response"] = response

In [399]:
response

ChatCompletion(id='chatcmpl-ARMqglJf1toesTEcCNSgWoNWx05fR', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dX6GQcpwAW62PtBEqsjHWJNM', function=Function(arguments='{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」 滋賀公演", "live_place": "滋賀U★STONE", "live_date": "2024/11/02"}}', name='get_live_info'), type='function'), ChatCompletionMessageToolCall(id='call_ssR8tFwQECj56F2euTdy37Hv', function=Function(arguments='{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」 京都公演", "live_place": "京都MUSE", "live_date": "2024/11/03"}}', name='get_live_info'), type='function'), ChatCompletionMessageToolCall(id='call_5pk0CHL3cpTmKglpti6r9p8I', function=Function(arguments='{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」 兵庫公演", "live_place": "神戸VARIT", "live_date": "2024/11/04"}}', name='get_live_info'), type='function'), ChatCompletionMessageToo

In [406]:
for tweet_id in tweets.keys():
    tweet_text = tweets[tweet_id]["tweet_text"]
    print(tweet_text)
    response = tweets[tweet_id]["gpt_response"]
    if (response.choices[0].finish_reason == "tool_calls"):
        for tool_call in response.choices[0].message.tool_calls:
            print(tool_call.function.arguments)
    print("====================")

【#花いろは 関西ツアー2024「いろはにほへと」 】

関西ツアーチケット詳細＆ゲスト発表

✦ː──────────

11/2(土)
＜滋賀＞滋賀U★STONE
OPEN17:00 / START17:30
https://tiget.net/events/347147
9/26(木)20:00〜順次

11/3(日)
＜京都＞京都MUSE 
OPEN17:00 / START17:30
ゲスト：なみだ色の消しゴム
 https://tiget.net/events/347359
9/26(木)21:00〜順次

11/4(月祝)
＜兵庫＞神戸VARIT 
OPNE11:30 / START12:00
ゲスト：KRD8
 https://tiget.net/events/347360
9/28(土)20:00〜順次

11/16(土)FINAL
＜大阪＞江坂MUSE 
OPEN18:00 / START18:30
ゲスト：サクヤコノハナ
 https://tiget.net/events/347361
9/28(土)21:00〜順次

✦ː──────────

関西・関東ツアー来場回数特典を予定しております！
ぜひゲストさんのファンの方も一緒に関西ツアー盛り上がっていきましょう
お待ちしております
{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」", "live_place": "滋賀U★STONE", "live_date": "2024/11/02"}}
{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」", "live_place": "京都MUSE", "live_date": "2024/11/03"}}
{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」", "live_place": "神戸VARIT", "live_date": "2024/11/04"}}
{"live_info": {"live_name": "花いろは 関西ツアー2024「いろはにほへと」", "live_place": "江坂MUSE", "live_date": "2024/11/16"}}
本日11/3

In [ ]:
response = get_live_info(prompt_user)